## Standard Continuous Time Lyapunov Equation

In [4]:
import scipy
import control
import numpy as np
import sympy as sp
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

from pydrake.all import (LinearQuadraticRegulator, MathematicalProgram, Variables,
                         Solve, RealContinuousLyapunovEquation)

import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (Jacobian, MathematicalProgram, Polynomial,
                         RegionOfAttraction, RegionOfAttractionOptions, Solve,
                         SymbolicVectorSystem, ToLatex, Variable, Variables,
                         plot_sublevelset_expression)
from pydrake.examples.pendulum import PendulumParams
from pydrake.examples.van_der_pol import VanDerPolOscillator
from pydrake.symbolic import Polynomial

from underactuated import plot_2d_phase_portrait, running_as_notebook

from pydrake.all import (AddMultibodyPlantSceneGraph, ControllabilityMatrix,
                         DiagramBuilder, Linearize, LinearQuadraticRegulator,
                         MeshcatVisualizerCpp, Parser, Saturation, SceneGraph,
                         Simulator, StartMeshcat, WrapToSystem)

if running_as_notebook:
    mpld3.enable_notebook()

import math

In [5]:
from pydrake.examples.pendulum import PendulumParams, PendulumPlant

p = PendulumParams()

b = p.damping()
m = p.mass()
length = p.length()
g = p.gravity()
grav = p.gravity()

## Simple Pendulum Example

Take the Lyapunov function at the vertical down position and verify that it will grow as the alternations converge. 

In [55]:
# explicit dynamics
plant = PendulumPlant()
context = plant.CreateDefaultContext()
plant.get_input_port().FixValue(context, [0])
context.SetContinuousState([0, 0])

linearized_plant = Linearize(plant, context)

A = linearized_plant.A()
Q = np.eye(2)
P = RealContinuousLyapunovEquation(A, Q)
X1 = control.lyap(A.T, Q)

In [22]:
#doing the van der pol oscillator implicitly

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + V.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
  
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(1e-5))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(1e-5), np.array(lambda_g_results)

def find_implicit_V(x, xd, lambda_, lambda_g, g, V_degree):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
    
    y = np.hstack([x, xd])
        
    V, Q = prog.NewSosPolynomial(Variables(x), V_degree)
    Vdot = V.Jacobian(x).dot(xd)
    
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
    
    prog.AddLinearCost(np.trace(Q))    
    #prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0, xd[0]: 0, xd[1]: 0}) == 0)
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0}) == 0)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("V solution status: " + str(k))
    
    assert result.is_success()
        
    return result.GetSolution(V), result.GetSolution(Q)


def problem_solver_implicit(S, V_degree):
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    xd = prog.NewIndeterminates(2, "xd")
        
    g0 = xd[0] - x[1]
    g1 = m*(length**2)*xd[1] + b*x[1] + m*grav*length*x[0]
    
    g = np.array([g0, g1])
    
    V = x.T@P@x
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(200):
        lambda_, lambda_g = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
#         print(alpha)
        
        V, Q = find_implicit_V(x, xd, lambda_, lambda_g, g_poly, V_degree)
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
    return V, Q

V_degree = 2
V, Q = problem_solver_implicit(P, V_degree)

$ V(x)=(0.050968400x_{0} x_{1} + 25.785193680x_{0}^{2} + 1.313710499x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.058812414x_{0} x_{1} + 25.148088222x_{0}^{2} + 1.279074892x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.063062259x_{0} x_{1} + 24.528546646x_{0}^{2} + 1.245323622x_{1}^{2})$

...

$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.000001360x_{0} x_{1} + 0.005291324x_{0}^{2} + 0.000269132x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 + 0.000000000x_{0} + 0.000000000x_{1} + 0.000001050x_{0} x_{1} + 0.004646298x_{0}^{2} + 0.000236327x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} + 0.000000000x_{1} + 0.000000834x_{0} x_{1} + 0.004055590x_{0}^{2} + 0.000206284x_{1}^{2})$

In [57]:
#THIS CODE WORKS
from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

def findLambda(x, V, Vdot, count):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    
    #V = Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression()
    
    lambda_ = prog.NewSosPolynomial(Variables(x), 2)[0].ToExpression()
        
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V))
    prog.AddSosConstraint(lambda_)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
        
    print('Status of first solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(lambda_)


def findV(x, lambda_, f, count):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    
    lambda_ = Polynomial(lambda_).RemoveTermsWithSmallCoefficients(1e-2).ToExpression()
        
    temp, Q = prog.NewSosPolynomial(Variables(x), 2)
    V = temp.ToExpression()
            
    prog.AddLinearConstraint(V.Substitute({x[0]: 0, x[1]:0}) == 0)
    
    Vdot = V.Jacobian(x).dot(f)
    
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V))
    
#     prog.AddSosConstraint(np.trace(Q))
    
    A = Q[0, 0]
    B = 2*Q[1, 0]
    C = Q[1, 1]
    
    prog.AddCost(A)
    prog.AddCost(C)
    prog.AddCost(-B)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
        
    k = result.get_solver_details().solution_status
                
    print('Status of second solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(V), result.GetSolution(Q)

def setup_solver():  
    count = 0
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    
    f0 = x[1]
    f1 = (-b/(m*length**2))*x[1] - (grav/length)*(x[0])
    
    f = np.array([f0, f1])
    
    V = x.T@P@x
    
    display(Markdown("$V(x) = " + ToLatex(
    Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
    #Vdot = x.T@P@A@x + x.T@A.T@P@x
    Vdot = V.Jacobian(x).dot(f)
    
    for i in range(20): 
        lambda_ = findLambda(x, V, Vdot, count)
        
#         display(Markdown("$\lambda (x) = " + ToLatex(
#         Polynomial(lambda_).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        V, Q = findV(x, lambda_, f, count)
        
        display(Markdown("$V(x) = " + ToLatex(
        Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        Vdot = V.Jacobian(x).dot(f)
        
        count += 1
    return V, Q
    
    
V, Q = setup_solver()

$V(x) = (0.050968x_{0} x_{1} + 25.785194x_{0}^{2} + 1.313710x_{1}^{2})$

Status of first solver on step 0: 1
Status of second solver on step 0: 1


$V(x) = (0.004926x_{0} x_{1} + 3.587715x_{0}^{2} + 0.182810x_{1}^{2})$

Status of first solver on step 1: 1
Status of second solver on step 1: 1


$V(x) = (0.002776x_{0} x_{1} + 2.291521x_{0}^{2} + 0.116767x_{1}^{2})$

...

$V(x) = (0.297182x_{0}^{2} + 0.015147x_{1}^{2})$

Status of first solver on step 18: 1
Status of second solver on step 18: 1


$V(x) = (0.266463x_{0}^{2} + 0.013581x_{1}^{2})$

Status of first solver on step 19: 1
Status of second solver on step 19: 1


$V(x) = 0$

### Pendulum Upright Example
Goal is to stabalize the pendulum in the upward position and certify the region of stability using Lyapunov analysis after we have found the controller.  

For this we first need to do a change or coordinates: $\tilde{\theta} = \theta - \pi$ 

The dynamics now become: 
$ml^2\ddot{\tilde{\theta}} + b\dot{\tilde{\theta}} - mgl\sin{\tilde{\theta}} = 0$

Here $\theta$ is the angle with the vertical when in the downward position as defined by default in Drake

In [65]:
p = PendulumParams()

# Add the Linearization
A = np.array([[0, 1], [p.mass()*p.gravity()*p.length(), -p.damping()]])
B = np.array([[0], [1]])
E = np.array([[1, 0], [0, p.mass()*p.length()*p.length()]])

# Add LQR Costs
Q = np.diag([10, 1])
R = [1]

# matrix needed for algorithm
S = np.array([[0], [0]])

In [66]:
# solve the generalized care
X, L, G = control.care(A, B, Q, R, S, E)

In [67]:
# G = R^-1B^TXE is the gain matrix to become A - BG for closed loop
G

array([[10.74101105,  2.42596626]])

To do Lyapunov analysis: close the loop and solve the linear Lyapunov equation. Then, use the dynamics to find the ROA

In [68]:
A_closed = A - B@G

Q_ly = np.eye(2)
P = control.lyap(A_closed.T, Q_ly, E = E.T)
candidate_matrix = E.T@P@E

u = -Gx. Remember to add this to the dynamics equations in the SOS program

In [81]:
# Use this as a starting candidate for ROA study

#doing the van der pol oscillator implicitly

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + V.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
  
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(1e-5))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(1e-5), np.array(lambda_g_results)

def find_implicit_V(x, xd, lambda_, lambda_g, g, V_degree):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
    
    y = np.hstack([x, xd])
        
    V, Q = prog.NewSosPolynomial(Variables(x), V_degree)
    Vdot = V.Jacobian(x).dot(xd)
    
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
    
    prog.AddLinearCost(np.trace(Q))    
    #prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0, xd[0]: 0, xd[1]: 0}) == 0)
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0}) == 0)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("V solution status: " + str(k))
    
    assert result.is_success()
        
    return result.GetSolution(V), result.GetSolution(Q)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    xd = prog.NewIndeterminates(2, "xd")
    
    u = -G@x
    sinx = x[0] - (x[0]**3)/6 
    
    g0 = xd[0] - x[1]
    g1 = p.mass()*(p.length()**2)*xd[1] + p.damping()*x[1] - p.mass()*p.gravity()*p.length()*sinx - u[0]
    
    g = np.array([g0, g1])
    
    V = x.T@S@x
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(200):
        lambda_, lambda_g = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
#         print(alpha)
        
        V, Q = find_implicit_V(x, xd, lambda_, lambda_g, g_poly, V_degree)
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
    return V, Q

V_degree = 2
V, Q = problem_solver_implicit(candidate_matrix, V_degree, G)

$ V(x)=(0.042837479x_{0} x_{1} + 1.421101750x_{0}^{2} + 0.051605869x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.056976379x_{0} x_{1} + 1.135005532x_{0}^{2} + 0.041001326x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 + 0.000000000x_{0} + 0.000000000x_{1} + 0.060421048x_{0} x_{1} + 0.925917515x_{0}^{2} + 0.031651785x_{1}^{2})$

...

$ V(x)=(0.000000000 - 0.000000000x_{0} + 0.000000000x_{1} + 0.001047690x_{0} x_{1} + 0.045779168x_{0}^{2} + 0.000875664x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 + 0.000000000x_{0} - 0.000000000x_{1} + 0.001045002x_{0} x_{1} + 0.045995233x_{0}^{2} + 0.000881428x_{1}^{2})$

Lambda solution status: 1
V solution status: 7


AssertionError: 

In [33]:
# explicit linearization
A_explicit = np.array([[0, 1], [p.gravity()/p.length(), -p.damping()/(p.mass()*p.length()*p.length())]])
B_explicit = np.array([[0], [1]])

Q_explicit = np.diag([10, 1])
R_explicit = [1]

K_explicit, S_explicit = LinearQuadraticRegulator(A_explicit, B_explicit, Q_explicit, R_explicit)

In [34]:
X_explicit, L_explicit, G_explicit = control.care(A_explicit, B_explicit, Q_explicit, R_explicit)

In [35]:
K_explicit

array([[39.4932081 ,  8.55245308]])

In [36]:
G_explicit

array([[39.4932081 ,  8.55245308]])

In [37]:
B_explicit.T@X_explicit

array([[39.4932081 ,  8.55245308]])

In [42]:
A_cl_ex = A_explicit - B_explicit@K_explicit
Q_ly_ex = np.eye(2)

In [43]:
P_ex = control.lyap(A_cl_ex.T, Q_ly_ex)

In [44]:
np.linalg.eigvals(P_ex)

array([1.39149567, 0.05818603])

In [52]:
P_ex

array([[1.39102074, 0.0251595 ],
       [0.0251595 , 0.05866096]])

In [53]:
candidate_matrix

array([[1.42110175, 0.02141874],
       [0.02141874, 0.05160587]])

In [64]:
#THIS CODE WORKS
from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

def findLambda(x, V, Vdot, count):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    
    #V = Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression()
    
    lambda_ = prog.NewSosPolynomial(Variables(x), 2)[0].ToExpression()
        
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V))
    prog.AddSosConstraint(lambda_)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
        
    print('Status of first solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(lambda_)


def findV(x, lambda_, f, count):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    
    lambda_ = Polynomial(lambda_).RemoveTermsWithSmallCoefficients(1e-2).ToExpression()
        
    temp, Q = prog.NewSosPolynomial(Variables(x), 2)
    V = temp.ToExpression()
            
    prog.AddLinearConstraint(V.Substitute({x[0]: 0, x[1]:0}) == 0)
    
    Vdot = V.Jacobian(x).dot(f)
    
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V))
    
#     prog.AddSosConstraint(np.trace(Q))
    
    A = Q[0, 0]
    B = 2*Q[1, 0]
    C = Q[1, 1]
    
    prog.AddCost(A)
    prog.AddCost(C)
    prog.AddCost(-B)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
        
    k = result.get_solver_details().solution_status
                
    print('Status of second solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(V), result.GetSolution(Q)

def setup_solver():  
    count = 0
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    
    f0 = x[1]
    f1 = (-b/(m*length**2))*x[1] + (grav/length)*(x[0])
    
    f = np.array([f0, f1])
    
    V = x.T@candidate_matrix@x
    
    display(Markdown("$V(x) = " + ToLatex(
    Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
    #Vdot = x.T@P@A@x + x.T@A.T@P@x
    Vdot = V.Jacobian(x).dot(f)
    
    for i in range(20): 
        lambda_ = findLambda(x, V, Vdot, count)
        
#         display(Markdown("$\lambda (x) = " + ToLatex(
#         Polynomial(lambda_).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        V, Q = findV(x, lambda_, f, count)
        
        display(Markdown("$V(x) = " + ToLatex(
        Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        Vdot = V.Jacobian(x).dot(f)
        
        count += 1
    return V, Q
    
    
V, Q = setup_solver()

$V(x) = (0.042837x_{0} x_{1} + 1.421102x_{0}^{2} + 0.051606x_{1}^{2})$

Status of first solver on step 0: 5


AssertionError: 